Universidad del Valle de Guatemala <br/>
Modelacion y Simulacion <br/>
Docente: Samuel Chavez <br/>
Semestre 2, 2020 <br/>

Dieter Esteban de Wit Torres 15146 <br/>
&nbsp;
&nbsp;

# Mini	proyecto 3
&nbsp;

In [1]:
# --- Import required libraries ---
import random
import math

In [2]:
# --- Important Definitions ---
def exponential_service(lamda):
    return -float((1/lamda)*math.log(random.random()))

def event_time_persistence(s, lamda):
    return s-(1/lamda)*math.log(random.random())

In [87]:
# --- Gorila MegaComputing Simulation ---
def GorillaMegaComputing(args):
    rate = 40
    closing_time = 3600
    request_times = []
    requests = 0
    exits = 0
    request_time = 0
    exit_time = 0
    processes = 0                  
    processing_time = 0
    total_time = 0
    inf = float('inf')
    
    while True:
        if(request_time <= exit_time and request_time <= closing_time):
            time = request_time
            requests = requests + 1
            processes = processes + 1
            request_time = event_time_persistence(time, rate)
            if processes == 1:
                Y = exponential_service(rate)
                exit_time = time + Y
            request_times.append(time)
        elif(exit_time < request_time and exit_time <= closing_time):
            time = exit_time
            processes = processes - 1
            exits = exits + 1
            if processes == 0:
                exit_time = inf
            else:
                Y = exponential_service(rate)
                exit_time = time + Y
                time = exit_time
        elif(min(request_time, exit_time) > closing_time and processes > 0):
            time = exit_time
            processes = processes - 1
            exits = exits + 1
            if processes > 0:
                Y = exponential_service(rate)
                exit_time = time + Y
                time = exit_time
        elif(min(request_time, exit_time) > closing_time and processes==0):
            processing_time = max(time - closing_time, 0)
            break
            
    if args == 'requests':
        return requests
    elif args == 'occupied':
        return exit_time
    elif args == 'free':
        return exit_time - request_time
    
    for req_time in range(len(request_times)):
        remaining_time = request_times[req_time]
        total_time = total_time + remaining_time
        
    if args == 'queue':
        return total_time/len(request_times)
    if args == 'prom_queue': 
        return (total_time/len(request_times))/requests

In [88]:
def AntsSmartComputing(servers, args):
    rate = 40
    time = 0                          
    closing_time = 3600                            
    request_times = []                              
    exit_times = []                              
    processing_times = []                              
    requests = 0                                                           
    request_time = exponential_service(rate)
    exit_time = [40] * servers
    total_time = 0
    sistem_requests = [0] * (1 + servers)
    server_processes = [0] * servers
    
    while time < closing_time or sistem_requests[0] > 0:           
        search_min_exit_time = exit_time.index(min(exit_time))            
        
        if request_time < exit_time[search_min_exit_time] and request_time < closing_time:   
            time = request_time                     
            requests = requests +  1                
            request_time = request_time + exponential_service(rate)
            request_times.append(time)                  
            
            if sistem_requests[0] == 0:              
                processing_times.append(time)             
                sistem_requests[0] += 1              
                sistem_requests[1] = requests              
                exit_time[0] = time + exponential_service(rate - 30) 
                
            elif sistem_requests[0] < servers:            
                available = sistem_requests.index(0) - 1 
                processing_times.append(time)             
                sistem_requests[0] += 1              
                sistem_requests[available + 1] = requests      
                exit_time[available] = time + exponential_service(rate - 30) 
            
            else:                      
                sistem_requests[0] += 1               
        else:                          
            time = exit_time[search_min_exit_time]             
            server_processes[search_min_exit_time] += 1            
            exit_times.append(time)                 
            
            if sistem_requests[0] <= servers:            
                sistem_requests[0] -= 1              
                exit_time[search_min_exit_time] = 1000000    
                sistem_requests[search_min_exit_time + 1] = 0      
            
            else:                       
                sistem_requests[0] *= -1
                next_process = max(sistem_requests) + 1  
                sistem_requests[0] *= -1
                processing_times.append(time)            
                sistem_requests[0] -= 1              
                exit_time[search_min_exit_time] = time + exponential_service(rate - 30) 
                sistem_requests[search_min_exit_time + 1] = next_process 
                
    if args == 'requests':
        return requests
    elif args == 'requests_detailed':
        print ("Solicitudes atendididas por cada servidor:")
        print ("-----------------------------------------")
        for process in range (len(server_processes)):
            print ("\t" + "Servidor " + str(process + 1) + ":", server_processes[process], "solicitudes.")
        print ("-----------------------------------------")
        return requests
    elif args == 'occupied':
        return exit_time
    elif args == 'free':
        return exit_time[0] - (exit_time[0] - request_time)
    
    for req_time in range(len(request_times)):
        remaining_time = processing_times[req_time] - request_times[req_time]
        total_time = total_time + remaining_time
        
    if args == 'queue':
        return total_time
    if args == 'prom_queue': 
        return total_time/requests
    
    print ("ultima llegada:", request_times[-1], "s.")
    print ("ultima salida:", exit_times[-1], "s.")

## Ejercicio	único – Servidores:

Es	usted	el/la	encargad@	de	definir	la	arquitectura	a	usar	en	el	lanzamiento	de	su	próxima	aplicación	Web:	C3 (Sistema	de	Contabilidad	del	Conde	Contador). La	junta	directiva	le	ha	solicitado	que	encuentre	el	mejor	servicio	de	hosting	para	el	proyecto.	Después	de	una	investigación	gigante,	usted	concluye	que	las	mejores	opciones	se	reducen	a	las	siguientes	dos:

### Proveedor	1	– Gorilla	Megacomputing:
    Tienen una infraestructura de servidor único, con mucha potencia de procesamiento. Ellos se enorgullecen al indicar que su servidor Enterprise puede atender hasta 100 solicitudes por segundo.

### Proveedor	2	– Ants	smart	computing:
    Tienen una infraestructura de múltiples servidores (en nube). Cada servidor es medianamente potente, y en su promoción indican que uno paga únicamente la cantidad de servidores que su aplicación requiera. Luego de su análisis de esta oferta, usted infiere que cada servidor tiene a lo sumo una décima parte de la potencia del servidor promocionado por Gorilla Megacomputing.

Las	pruebas	de	estrés	iniciales,	y	las	proyecciones	calculadas	para	los	primeros	2	años	luego	del	lanzamiento,	indican	que	su	aplicación	jamás	excederá	los	2400	solicitudes	por	minuto.	Una	auditoría	y	análisis	de	benchmark a	sistemas	similares	al	suyo,	indican	que	las	solicitudes	deberían	llegar como	un	proceso	de	Poisson,	y	que	el	tiempo	de	servicio	de	cada	solicitud	(sin	importar	la	arquitectura	de	servidor	usada)	es	modelado	adecuadamente	por	una	variable	aleatoria	exponencial.

Mañana	tiene	que	presentar	su	decisión	final	a	la	junta	directiva	del	proyecto.		Como	no	tiene	tiempo	para	hacer una	investigación	a	detalle	con	los	clientes	de	cada	proveedor,	decide	creer	en	su	promoción	y	hacer	una	simulación	para	concluir	cuál	será	la	mejor	opción.

### Task 1: 

Modele,	simule	y	analice el	comportamiento	de	ambos	sistemas	durante	una	hora	de	ejecución	de	C3,	y	para	cada	sistema	responda:

- ¿Cuántas	solicitudes atendió	cada	servidor?

In [64]:
print ("> Gorilla MegaComputing atendio", GorillaMegaComputing('requests'), "solicitudes.")
print ("> Ants SmartComputing atendio", AntsSmartComputing(10, 'requests'), "solicitudes.")

> Gorilla MegaComputing atendio 143806 solicitudes.
> Ants SmartComputing atendio 144553 solicitudes.


- ¿Cuánto	tiempo	estuvo	cada	servidor	ocupado?

In [81]:
ants_occupied_seconds = AntsSmartComputing(10, 'occupied')

print ("> Gorilla MegaComputing se mantuvo ocupado por", GorillaMegaComputing('occupied'), "segundos.")
print ("> Ants SmartComputing se mantuvo ocupado por: \n ", ants_occupied_seconds[0], "segundos por servidor. \n ", sum(ants_occupied_seconds), "segundos totales.")

> Gorilla MegaComputing se mantuvo ocupado por 3604.1232479359314 segundos.
> Ants SmartComputing se mantuvo ocupado por: 
  1000000 segundos por servidor. 
  10000000 segundos totales.


- ¿Cuánto	tiempo	estuvo	cada	servidor	desocupado	(idle)?

In [85]:
ants_idle_seconds = AntsSmartComputing(10, 'free')

print ("> Gorilla MegaComputing se mantuvo desocupado por", GorillaMegaComputing('free'), "segundos.")
print ("> Ants SmartComputing se mantuvo desocupado por: \n ", ants_idle_seconds/10, "segundos por servidor aproximadamente. \n ", ants_idle_seconds, "segundos totales")

> Gorilla MegaComputing se mantuvo desocupado por 10.660237456908362 segundos.
> Ants SmartComputing se mantuvo desocupado por: 
  360.00077753331277 segundos por servidor aproximadamente. 
  3600.007775333128 segundos totales


- ¿Cuánto	tiempo	en	total	estuvieron	las	solicitudes en	cola?

In [89]:
ants_queue_seconds = AntsSmartComputing(10, 'queue')

print ("> Se mantuvieron en cola las solicitudes de Gorilla MegaComputing por", GorillaMegaComputing('queue'), "segundos.")
print ("> Se mantuvieron en cola las solicitudes de Ants SmartComputing por: \n ", ants_queue_seconds, "segundos por servidor aproximadamente. \n ", ants_queue_seconds*10, "segundos totales.")

> Se mantuvieron en cola las solicitudes de Gorilla MegaComputing por 1795.9954288544525 segundos.
> Se mantuvieron en cola las solicitudes de Ants SmartComputing por: 
  21.266438117234856 segundos por servidor aproximadamente. 
  212.66438117234856 segundos totales.


- En	promedio	¿cuánto	tiempo	estuvo	cada	solicitud en	cola?

In [91]:
ants_prom_queue_seconds = AntsSmartComputing(10, 'prom_queue')

print ("> En promedio, estuvo cada solicitud en cola en Gorilla MegaComputing por", GorillaMegaComputing('prom_queue'), "segundos aproximadament.")
print ("> En promedio, estuvo cada solicitud en cola en Ants SmartComputing por", ants_prom_queue_seconds, "segundos por servidor aproximadamente.")

> En promedio, estuvo cada solicitud en cola en Gorilla MegaComputing por 0.012473045565797168 segundos aproximadament.
> En promedio, estuvo cada solicitud en cola en Ants SmartComputing por 0.00015737033667640585 segundos por servidor aproximadamente.


### Task 2: 

Determine	experimentalmente	**p1**,	**p2**,	**p3** que	hacen	su	dibujo	más	denso.

- La probabilidad individual para **p1**, **p2** y **p3** que genera el dibujo mas denso es de 0.333 para cada una de ellas, sin embargo no se represento en el codigo debido a que como la probabilidad es la misma para las 3 individuales no existe cambio si se multiplica el estado deterministico or un factor unitario. 